# Property Graph Index with Local Llama Model and Local File

In this notebook, we demonstrate how to create, query, and customize a Property Graph Index using a local Llama model and a local file.

In [1]:
!pip install llama-index
!pip install llama-index-embeddings-ollama

## Setting Up the Environment

In [2]:
import os
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

## Loading Documents

In [4]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
#documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()

# Load documents from the local directory
print(f"Number of documents loaded: {len(documents)}")

Number of documents loaded: 1


## Constructing the Property Graph Index

In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
from llama_index.core import PropertyGraphIndex

# Create Property Graph Index
index = PropertyGraphIndex.from_documents(
    documents,
    llm=ollama_llm,
    embed_model=ollama_embedding,
    show_progress=True
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 11/11 [00:01<00:00,  8.00it/s]


## Exploring the Property Graph

In [9]:
!pip install pyvis


   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   --------------------------- ------------ 524.3/756.0 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 756.0/756.0 kB 5.3 MB/s eta 0:00:00


In [10]:

# Save and visualize the graph
index.property_graph_store.save_networkx_graph(name="./kg.html")
print("Graph saved as ./kg.html. Open it in a browser to explore.")

Graph saved as ./kg.html. Open it in a browser to explore.


## Querying the Property Graph

In [12]:
# Simple Retrieval
retriever = index.as_retriever(include_text=False)

# Query the graph
nodes = retriever.retrieve("What happened at Interleaf and Viaweb?")
for node in nodes:
    print(node.text)

Interleaf -> Was on the way down -> And i didn't want to work there
Viaweb -> Started -> After the web-based store builder worked
Robert morris -> Showed me -> The world wide web
Paul graham -> Was -> At the height of the internet bubble


In [14]:
# Advanced Querying
query_engine = index.as_query_engine(include_text=True, llm=ollama_llm,)

response = query_engine.query("What happened at Interleaf and Viaweb?")
print(str(response))

Interleaf was a company where Paul Graham worked as a freelance Lisp hacker. He didn't want to work there because he sensed that it was on the way down. 

Viaweb started after the web-based store builder worked, which was initially built for art galleries but later realized that online stores were more in demand and could be built using similar technology. The company's software eventually evolved into a web app, where users could control the site through a browser instead of needing client software or typing commands on the server.


## Persisting the Property Graph Index

In [15]:
# Save the graph and embeddings for reuse
index.storage_context.persist(persist_dir="./property_graph_storage")

In [19]:
# Configure Ollama Embedding
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

In [27]:

from llama_index.core import StorageContext,ServiceContext, load_index_from_storage
from llama_index.core import Settings

# Set global settings
Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding


# Create a StorageContext
storage_context = StorageContext.from_defaults(
    persist_dir="./property_graph_storage"
)

# Load the index with service_context and storage_context
loaded_index = load_index_from_storage(
    storage_context=storage_context,
)

print("Index loaded successfully with ServiceContext!")


Index loaded successfully with ServiceContext!


In [28]:
# Simple Retrieval
retriever = loaded_index.as_retriever(include_text=False)

# Query the graph
nodes = retriever.retrieve("What happened at Interleaf and Viaweb?")
for node in nodes:
    print(node.text)

Interleaf -> Was on the way down -> And i didn't want to work there
Viaweb -> Started -> After the web-based store builder worked
Robert morris -> Showed me -> The world wide web
Paul graham -> Was -> At the height of the internet bubble
